# Wordcloud
<br>  

### References
- [jqassistant](https://jqassistant.org)
- [py2neo](https://py2neo.org/2021.1/)

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plot
from py2neo import Graph
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
# Please set the environment variable "NEO4J_INITIAL_PASSWORD" in your shell 
# before starting jupyter notebook to provide the password for the user "neo4j". 
# It is not recommended to hardcode the password into jupyter notebook for security reasons.
graph = Graph("bolt://localhost:7687", auth=("neo4j", os.environ.get("NEO4J_INITIAL_PASSWORD")))

In [ ]:
def get_cypher_query_from_file(filename):
    with open(filename) as file:
        return ' '.join(file.readlines())

In [ ]:
def query_cypher_to_data_frame(filename):
    return graph.run(get_cypher_query_from_file(filename)).to_data_frame()

In [ ]:
#The following cell uses the build-in %html "magic" to override the CSS style for tables to a much smaller size.
#This is especially needed for PDF export of tables with multiple columns.

In [ ]:
%%html
<style>
/* CSS style for smaller dataframe tables. */
.dataframe th {
    font-size: 8px;
}
.dataframe td {
    font-size: 8px;
}
</style>

In [ ]:
# Main Colormap
main_color_map = 'nipy_spectral'

## Word Cloud

In [ ]:
# Query data from graph database
words = query_cypher_to_data_frame("../cypher/Overview/Words_for_Wordcloud.cypher")
words.head(20)

In [ ]:
# Join all words into one text separated by spaces
text = " ".join(i for i in words.word)
print("There are {} words in the dataset.".format(len(words.word)))

# Define stop words
stopwords = set(STOPWORDS)
stopwords.update(['builder', 'exception', 'abstract', 'helper', 'util', 'callback', 'factory', 'handler', 'repository', 'result'])
wordcloud = WordCloud(stopwords=stopwords, background_color='white', colormap='viridis').generate(text)

# Plot the word cloud
plot.figure(figsize=(15,10))
plot.imshow(wordcloud, interpolation='bilinear')
plot.axis("off")
plot.show()